In [1]:
# Set up deterministic flag
import tensorflow as tf
import os
os.environ["TF_DETERMINISTIC_OPS"] = "1"

In [2]:
# Fix random seeds
SEED = 666
tf.random.set_seed(SEED)
import numpy as np
np.random.seed(SEED)

In [3]:
# Import wandb
import wandb
from wandb.keras import WandbCallback

In [4]:
# Other imports
import time

In [5]:
# Load up and preprocess data
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

In [6]:
# Verify shapes
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [7]:
# Specify the class labels
LABELS = ["T-shirt/top","Trouser","Pullover","Dress","Coat",
        "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

In [8]:
# Create a sepcific validation set from the test set
X_val = []
y_val = []

idx = np.random.choice(test_images.shape[0], 32)
for i in idx:
    X_val.append(test_images[i])
    y_val.append(test_labels[i])
    
X_val, y_val = np.array(X_val), np.array(y_val)

In [9]:
# Verify shapes
X_val.shape, y_val.shape

((32, 28, 28), (32,))

In [10]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
# Reference: https://colab.research.google.com/drive/181GCGp36_75C2zm7WLxr9U2QjMXXoibt#scrollTo=xftEwKyuaJ5q
sweep_config = {
    "method": "random", #grid, random
    "metric": {
      "name": "accuracy",
      "goal": "maximize"   
    },
    "parameters": {
        "epochs": {
            "values": [10, 15, 20]
        },
        "learning_rate": {
            "values": [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        },
        "optimizer": {
            'values': ["adam", "sgd"]
        }
    }
}

In [17]:
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, entity="sayakpaul", project="reproducible-ml")

Create sweep with ID: 6t8t4d6s
Sweep URL: https://app.wandb.ai/sayakpaul/reproducible-ml/sweeps/6t8t4d6s


In [12]:
# Utility function for model
def get_training_model(prefinal_act, final_act, opt):
    # Define the model
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(256, activation=prefinal_act),
        tf.keras.layers.Dense(10, activation=final_act)
    ])
    
    # Compile the model
    model.compile(optimizer=opt,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

In [13]:
# Set up early stopping callback
es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                      patience=1,
                                      restore_best_weights=True,
                                      verbose=3)

In [14]:
# Define the training loop
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        "epochs": 10,
        "batch_size": 128,
        "prefinal_activation": "relu",
        "final_activation": "softmax",
        "optimizer": "adam",
        'seed': 42
    }
    
    # Initialize a new wandb run
    wandb.init(config=config_defaults, project="reproducible-ml")
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    
    # Call model with the current parameters
    mlp_model = get_training_model(config.prefinal_activation, config.final_activation, config.optimizer)
    
    start = time.time()
    mlp_model.fit(train_images, train_labels, 
        validation_data=(test_images, test_labels),
        batch_size=config.batch_size,
        epochs=config.epochs,
        callbacks=[WandbCallback(data_type="image", validation_data=(X_val, y_val), 
                        labels=LABELS),
                  es])
    wandb.log({"training_time":time.time()-start})

In [ ]:
# Start the sweep
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 0ggst34b with config:
	epochs: 15
	learning_rate: 1e-05
	optimizer: sgd
wandb: Agent Started Run: 0ggst34b


Error generating diff: Reference at 'refs/remotes/origin/master' does not exist


Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 2s 39us/sample - loss: 1.0310 - accuracy: 0.6870 - val_loss: 0.7503 - val_accuracy: 0.7553
Epoch 2/15
60000/60000 [==============================] - 1s 24us/sample - loss: 0.6687 - accuracy: 0.7862 - val_loss: 0.6417 - val_accuracy: 0.7864
Epoch 3/15
60000/60000 [==============================] - 1s 23us/sample - loss: 0.5911 - accuracy: 0.8082 - val_loss: 0.5869 - val_accuracy: 0.8050
Epoch 4/15
60000/60000 [==============================] - 1s 23us/sample - loss: 0.5491 - accuracy: 0.8191 - val_loss: 0.5601 - val_accuracy: 0.8074
Epoch 5/15
60000/60000 [==============================] - 1s 23us/sample - loss: 0.5216 - accuracy: 0.8250 - val_loss: 0.5358 - val_accuracy: 0.8149
Epoch 6/15
60000/60000 [==============================] - 1s 23us/sample - loss: 0.5025 - accuracy: 0.8312 - val_loss: 0.5254 - val_accuracy: 0.8183
Epoch 7/15
60000/60000 [==============================] 

Error generating diff: Reference at 'refs/remotes/origin/master' does not exist


Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 2s 40us/sample - loss: 0.5257 - accuracy: 0.8180 - val_loss: 0.4436 - val_accuracy: 0.8460
Epoch 2/20
60000/60000 [==============================] - 1s 24us/sample - loss: 0.3913 - accuracy: 0.8607 - val_loss: 0.3942 - val_accuracy: 0.8611
Epoch 3/20
60000/60000 [==============================] - 1s 24us/sample - loss: 0.3493 - accuracy: 0.8747 - val_loss: 0.3751 - val_accuracy: 0.8612
Epoch 4/20
60000/60000 [==============================] - 1s 24us/sample - loss: 0.3207 - accuracy: 0.8842 - val_loss: 0.3597 - val_accuracy: 0.8716
Epoch 5/20
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3072 - accuracy: 0.8881 - val_loss: 0.3488 - val_accuracy: 0.8747
Epoch 6/20
60000/60000 [==============================] - 1s 24us/sample - loss: 0.2888 - accuracy: 0.8938 - val_loss: 0.3519 - val_accuracy: 0.8749
Epoch 00006: early stopping
wandb: Agent Finished Run: 0

...

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 2s 34us/sample - loss: 1.0310 - accuracy: 0.6870 - val_loss: 0.7503 - val_accuracy: 0.7553
Epoch 2/20
60000/60000 [==============================] - 1s 22us/sample - loss: 0.6687 - accuracy: 0.7862 - val_loss: 0.6417 - val_accuracy: 0.7864
Epoch 3/20
60000/60000 [==============================] - 1s 22us/sample - loss: 0.5911 - accuracy: 0.8082 - val_loss: 0.5869 - val_accuracy: 0.8050
Epoch 4/20
60000/60000 [==============================] - 1s 21us/sample - loss: 0.5491 - accuracy: 0.8191 - val_loss: 0.5601 - val_accuracy: 0.8074
Epoch 5/20
60000/60000 [==============================] - 1s 22us/sample - loss: 0.5216 - accuracy: 0.8250 - val_loss: 0.5358 - val_accuracy: 0.8149
Epoch 6/20
60000/60000 [==============================] - 1s 22us/sample - loss: 0.5025 - accuracy: 0.8312 - val_loss: 0.5254 - val_accuracy: 0.8183
Epoch 7/20
60000/60000 [==============================] 

Error generating diff: Reference at 'refs/remotes/origin/master' does not exist


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.5257 - accuracy: 0.8180 - val_loss: 0.4436 - val_accuracy: 0.8460
Epoch 2/10
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3913 - accuracy: 0.8607 - val_loss: 0.3942 - val_accuracy: 0.8611
Epoch 3/10
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3493 - accuracy: 0.8747 - val_loss: 0.3751 - val_accuracy: 0.8612
Epoch 4/10
60000/60000 [==============================] - 1s 22us/sample - loss: 0.3207 - accuracy: 0.8842 - val_loss: 0.3597 - val_accuracy: 0.8716
Epoch 5/10
60000/60000 [==============================] - 1s 22us/sample - loss: 0.3072 - accuracy: 0.8881 - val_loss: 0.3488 - val_accuracy: 0.8747
Epoch 6/10
60000/60000 [==============================] - 1s 22us/sample - loss: 0.2888 - accuracy: 0.8938 - val_loss: 0.3519 - val_accuracy: 0.8749
Epoch 00006: early stopping
wandb: Agent Finished Run: q

Error generating diff: Reference at 'refs/remotes/origin/master' does not exist


Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 2s 35us/sample - loss: 0.5257 - accuracy: 0.8180 - val_loss: 0.4436 - val_accuracy: 0.8460
Epoch 2/15
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3913 - accuracy: 0.8607 - val_loss: 0.3942 - val_accuracy: 0.8611
Epoch 3/15
60000/60000 [==============================] - 1s 22us/sample - loss: 0.3493 - accuracy: 0.8747 - val_loss: 0.3751 - val_accuracy: 0.8612
Epoch 4/15
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3207 - accuracy: 0.8842 - val_loss: 0.3597 - val_accuracy: 0.8716
Epoch 5/15
57728/60000 [===========================>..] - ETA: 0s - loss: 0.3075 - accuracy: 0.8879